# SQL

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import OpenAI, SelfAskWithSearchChain, LLMMathChain
from sqlalchemy.engine import Engine, create_engine
from langchain.agents import initialize_agent, Tool
import os

In [3]:
import sys
 # setting path
sys.path.append('..')
sys.path.append('../modules')

from modules.sql_chain import SQLDatabaseChain
from modules.sql_database import SQLDatabase

In [4]:
# connect to remote postgres database
db = SQLDatabase.from_uri(os.getenv("DB_CONNECTION_STRING"))
print("Tables found:", db._all_tables)
db_chain = SQLDatabaseChain(llm=OpenAI(), database=db, verbose=True)

Tables found: ['users', 'templates', 'models', 'generations']
type <class 'modules.sql_database.SQLDatabase'>


In [5]:
# Load the tool configs that are needed.
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about data in your database"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [6]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [8]:
agent.run("We had a bug yesterday. Are there any users that are stuck in training?")



> Entering new ZeroShotAgent chain...
We had a bug yesterday. Are there any users that are stuck in training?
Thought: I need to find out if any users are stuck in training.
Action: Database
Action Input: Search for users in training

> Entering new SQLDatabaseChain chain...
Search for users in training 
SQLQuery: SELECT * FROM users WHERE payment_status = 'training';
SQLResult: []
Answer: There are no users in training.
> Finished SQLDatabaseChain chain.

Observation:  There are no users in training.
Thought: I now know the final answer.
Final Answer: No, there are no users stuck in training.
> Finished ZeroShotAgent chain.


'No, there are no users stuck in training.'